---
title: Redfield Issue check "Analytically"
date: 2024-08-08
authors:
  - name: Gerardo Suarez
---

Since there seems to be an issue with the Bloch-Redfield Solver in qutip and my not so good implementation 
of the time dependent redfield, here's a quick Analytical check to make sure it's not, by solving the
equation for the SYK(2) model symbolically. The Hamiltonian is given by

In [31]:
from sympy import *
from sympy.physics.quantum import Dagger
from sympy import I
from collections import defaultdict
import numpy as np
import itertools
from sympy.solvers.ode.systems import dsolve_system

In [73]:
a = symbols('a',real=True,positive=True)

In [74]:
H = diag(-a,a,a,-a)#the times 20 is to rescale time
Eq(S('H'), H, evaluate=False)

Eq(H, Matrix([
[-a, 0, 0,  0],
[ 0, a, 0,  0],
[ 0, 0, a,  0],
[ 0, 0, 0, -a]]))

In [38]:
b= symbols('b0:8')

Here I define the coupling operator $q$ that make things break for Bloch-Redfield on the SYK model

In [39]:
# coupling operator non trivial like in the shinsey ryu paper. This difference is not dramatic for simpler couplings
Q = Matrix([[0, b[0] , b[1], 0],
              [conjugate(b[0]), 0, 0, -b[1]],
              [conjugate(b[1]), 0, 0, b[0] ],
              [0, -conjugate(b[1]), conjugate(b[0]) , 0]])

In [40]:
Eq(S("q"), Q, evaluate=False)

Eq(q, Matrix([
[            0,             b0,            b1,   0],
[conjugate(b0),              0,             0, -b1],
[conjugate(b1),              0,             0,  b0],
[            0, -conjugate(b1), conjugate(b0),   0]]))

In [41]:
Dagger(Q) == Q #checking I didn't mess up 

True

In [42]:
def comm(a,b):
    return (a*b-b*a)

I then get the eigenvalues and eigenvectors to obtain the jump operators (this steps are hidden in the pdf)

In [43]:
basis=H.eigenvects() # I probably don't need this step since the Hamiltonian is
# Diagonal
energies=[[i[0]]*i[1] for i in basis]
energies = [x for xs in energies for x in xs]
states = [i[2] for i in basis]
states = [x for xs in states for x in xs ]

In [44]:
def jump_operators(H,evals,all_state, Q):
    """
    A function to obtain the jump operators
    """
    collapse_list = []
    ws = []
    N=len(energies)
    for j in range(N):
        for k in range(j + 1, N):
            Deltajk = evals[k] - evals[j]
            ws.append(Deltajk)
            collapse_list.append(
                (
                    all_state[j]
                    * Dagger(all_state[j])
                    * Q
                    * all_state[k]
                    * Dagger(all_state[k])
                )
            )  # emission
            ws.append(-Deltajk)
            collapse_list.append(
                (
                    all_state[k]
                    * Dagger(all_state[k])
                    * Q
                    * all_state[j]
                    * Dagger(all_state[j])
                )
            )  # absorption
    collapse_list.append(Q - sum(collapse_list,zeros(N)))  # Dephasing
    ws.append(0)
    output = defaultdict(list)
    for k, key in enumerate(ws):
        output[key].append(collapse_list[k])
    eldict = {x: sum(y, zeros(N)) for x, y in output.items()}
    dictrem = {}
    empty = 0*H
    for keys, values in eldict.items():
        if not (values == empty):
            dictrem[keys] = values
    return dictrem

In [45]:
jumps=jump_operators(H,energies,states,Q)

In [46]:
for key,value in jumps.items():
    print(latex(Eq(S(f"A({key})"), value, evaluate=False)))

A{\left(2 a \right)} = \left[\begin{matrix}0 & b_{0} & b_{1} & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & - \overline{b_{1}} & \overline{b_{0}} & 0\end{matrix}\right]
A{\left(- 2 a \right)} = \left[\begin{matrix}0 & 0 & 0 & 0\\\overline{b_{0}} & 0 & 0 & - b_{1}\\\overline{b_{1}} & 0 & 0 & b_{0}\\0 & 0 & 0 & 0\end{matrix}\right]


### Jump operator checks

The jump operators must satisfy

$$[H,A(\omega)]=-\omega A(\omega)$$
$$[H,A^{\dagger}(\omega) A(\omega)]=0$$
$$\sum_{w} A(\omega)= A$$


In [13]:
comm(H,jumps[2*a])/(-2*a) == jumps[2*a] #checking jump decomposition is fine

True

In [14]:
comm(H,Dagger(jumps[2*a])*jumps[2*a]) == zeros(4)

True

In [15]:
jumps[2*a]+jumps[-2*a]==Q

True

### Constucting the Differential equations
We now construct the differential equations from the GKLS form of the bloch 
Redfield generator

$$\begin{aligned}
     \dot{\rho_{S}^{I}(t)}{t} &=
    \sum_{\omega,\omega',\alpha,\beta} \gamma_{\beta,\alpha}(\omega,\omega')  \left( S_{\alpha}(\omega')  \rho^{I}_{S}(t) S^{\dagger}_{\beta}(\omega)- \frac{ \{S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega'),\rho^{I}_{S}(t)\}}{2} \right) \nonumber \\
    &+i \sum_{\omega,\omega',\alpha,\beta} S_{\beta,\alpha}(\omega,\omega') \Big[ \rho^{I}_{S}(t), S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega')\Big]\end{aligned}$$

I solve in the interaction picture generally, I did the same in my numerics so it should not be an issue (I rotate in the end).
By neglecting Lambshift as in the numerics

$$\dot{\rho_{S}^{I}(t)}{t} =
    \sum_{\omega,\omega',\alpha,\beta} \gamma_{\beta,\alpha}(\omega,\omega')  \left( S_{\alpha}(\omega')  \rho^{I}_{S}(t) S^{\dagger}_{\beta}(\omega)- \frac{ \{S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega'),\rho^{I}_{S}(t)\}}{2} \right)$$


In [16]:
# Simply define the symbolic density operator
rho = symbols('rho_1:17')
rho = Matrix(
    [rho[0: 4],
     rho[4: 8],
     rho[8: 12],
     rho[12:]])
Eq(S('rho'), rho, evaluate=False)


Eq(rho, Matrix([
[ rho_1,  rho_2,  rho_3,  rho_4],
[ rho_5,  rho_6,  rho_7,  rho_8],
[ rho_9, rho_10, rho_11, rho_12],
[rho_13, rho_14, rho_15, rho_16]]))

In [17]:
# This bit is simply the GKLS form without the appropiate coefficients
def matrix_form(jumps, combinations,rho):
    matrixform = {}
    for i in combinations:
        matrixform[i] =  (
            jumps[i[1]]*rho *Dagger(jumps[i[0]]) -
            (0.5 *
                (Dagger(jumps[i[0]]) * jumps[i[1]]*rho +
                rho*Dagger(jumps[i[0]])* jumps[i[1]])))
    return matrixform

As the sum goes on $(\omega,\omega')$ pairs I construct all combinations

In [18]:
ws = list(jumps.keys())
combinations = list(itertools.product(ws, ws))
combinations

[(2*a, 2*a), (2*a, -2*a), (-2*a, 2*a), (-2*a, -2*a)]

I get the GKLS form of each of those combinations, as a dictionary

In [19]:
matrices=matrix_form(jumps,combinations,rho)

In [20]:
(jumps[(2*a)]*jumps[-2*a]*rho+ rho*jumps[(2*a)]*jumps[-2*a]).expand()

Matrix([
[  2*b0*rho_1*conjugate(b0) + 2*b1*rho_1*conjugate(b1),   b0*rho_2*conjugate(b0) + b1*rho_2*conjugate(b1),   b0*rho_3*conjugate(b0) + b1*rho_3*conjugate(b1),   2*b0*rho_4*conjugate(b0) + 2*b1*rho_4*conjugate(b1)],
[      b0*rho_5*conjugate(b0) + b1*rho_5*conjugate(b1),                                                 0,                                                 0,       b0*rho_8*conjugate(b0) + b1*rho_8*conjugate(b1)],
[      b0*rho_9*conjugate(b0) + b1*rho_9*conjugate(b1),                                                 0,                                                 0,     b0*rho_12*conjugate(b0) + b1*rho_12*conjugate(b1)],
[2*b0*rho_13*conjugate(b0) + 2*b1*rho_13*conjugate(b1), b0*rho_14*conjugate(b0) + b1*rho_14*conjugate(b1), b0*rho_15*conjugate(b0) + b1*rho_15*conjugate(b1), 2*b0*rho_16*conjugate(b0) + 2*b1*rho_16*conjugate(b1)]])

Then I construct the generator by multiplying the appropiate coefficient to each of the GKLS from matrices. Now for the coefficients we have

$$\begin{aligned}
    \Gamma_{\alpha,\beta}(\omega,t) = \int_{0}^{t} ds e^{i \omega s}  \langle B^{\dagger}_{\alpha}(t) B_{\beta}^{\dagger}(t-s) \rangle_{B} \end{aligned}$$


Since I mainly care about bloch-redfield I make $t \xrightarrow{} \infty$ (in the integral) so

$$\begin{aligned}
    \Gamma_{\alpha,\beta}(\omega,\omega',t) =  \int_{0}^{\infty} ds e^{i \omega s}  \langle B(s) B(0) \rangle_{B} = \Gamma_{\alpha,\beta}(\omega) \end{aligned}$$

Where $\Gamma_{\alpha,\beta}(\omega)$  is the power spectrum


The decay rate in the redfield equation is given by

$\gamma(\omega,\omega',t)= e^{i(\omega-\omega')t}(\Gamma(\omega')+\overline{\Gamma(\omega)})$


In [21]:
lam,T,w0,w,t,gam=symbols("lambda T w0 w t gamma",real=True)

In [22]:
gmm,gpp=symbols("\\gamma_{--} \\gamma_{++}",real=True)
gpm=symbols("\\gamma_{+-}")


In [23]:
def power_spectrum(s):
    if (s[0]<0) and (s[1]<0):
        return gmm
    elif (s[0]>0) and (s[1]>0) :
        return gpp
    elif s[0]>0:
        return gpm
    else: 
        return conjugate(gpm)

Assuming an underdamped spectral density 

$$J(\omega)=\frac{\lambda^2  \gamma  \omega}{ ((\omega_{0}^2 - \omega^2)^2 + \gamma^2  \omega^2)} $$

In [24]:
def generator(combinations, matrices):
    gen = [-I*(H*rho-rho*H)+
        power_spectrum(s) * matrices[s]
        for s in combinations]
    return sum(gen, zeros(4))      

In [25]:
gene = generator(combinations,matrices)

Then the generator of the dynamics is given by

In [26]:
Eq(S('G'), gene, evaluate=False)

Eq(G, Matrix([
[                                                   \gamma_{++}*((b0*rho_6 + b1*rho_10)*conjugate(b0) + (b0*rho_7 + b1*rho_11)*conjugate(b1)) + \gamma_{--}*(-0.5*rho_1*(b0*conjugate(b0) + b1*conjugate(b1)) - 0.5*(b0*rho_1 - rho_4*conjugate(b1))*conjugate(b0) - 0.5*(b1*rho_1 + rho_4*conjugate(b0))*conjugate(b1)),                                                    \gamma_{++}*(-0.5*b0*(rho_2*conjugate(b0) + rho_3*conjugate(b1)) + 0.5*(b0*rho_3 - b1*rho_2)*conjugate(b1)) - 0.5*\gamma_{--}*rho_2*(b0*conjugate(b0) + b1*conjugate(b1)) + 8*I*a*rho_2 + (b0*(b0*rho_5 + b1*rho_9) - (b0*rho_8 + b1*rho_12)*conjugate(b1))*conjugate(\gamma_{+-}),                                                    \gamma_{++}*(-0.5*b1*(rho_2*conjugate(b0) + rho_3*conjugate(b1)) - 0.5*(b0*rho_3 - b1*rho_2)*conjugate(b0)) - 0.5*\gamma_{--}*rho_3*(b0*conjugate(b0) + b1*conjugate(b1)) + 8*I*a*rho_3 + (b1*(b0*rho_5 + b1*rho_9) + (b0*rho_8 + b1*rho_12)*conjugate(b0))*conjugate(\gamma_{+-}),                  

Next we simply vectorize the density matrix and construct the system of ODES

In [27]:
mgene = Matrix(list(gene))
rhom=Matrix(list(rho))

In [28]:
rhof = symbols('rho_1:17',cls=Function) # This is necessary to solve the differential equation
to_funcs={i:rhof[k](t) for k,i in enumerate(rhom)} #could have done so from the beginning though

In [29]:
mgene=mgene.subs(to_funcs) #substitute symbols for a function
mrhof=rhom.subs(to_funcs)

In [30]:
eqs = [Eq(mrhof[k].diff(t), mgene[k].expand().subs(b[0]*conjugate(b[0]),abs(b[0])**2).subs(b[1]*conjugate(b[1]),abs(b[1])**2).collect(0.5).nsimplify().collect(gpp).collect(gpm).collect(gmm).collect(conjugate(gpm))) for k in range(16)]

In [31]:
for i in eqs[12:]:
    print(latex(i))

\frac{d}{d t} \rho_{13}{\left(t \right)} = \gamma_{++} \left(\rho_{10}{\left(t \right)} \overline{b_{0}}^{2} + \rho_{11}{\left(t \right)} \overline{b_{0}} \overline{b_{1}} - \rho_{6}{\left(t \right)} \overline{b_{0}} \overline{b_{1}} - \rho_{7}{\left(t \right)} \overline{b_{1}}^{2}\right) + \gamma_{--} \left(- \rho_{13}{\left(t \right)} \left|{b_{0}}\right|^{2} - \rho_{13}{\left(t \right)} \left|{b_{1}}\right|^{2}\right)
\frac{d}{d t} \rho_{14}{\left(t \right)} = \gamma_{++} \left(- \frac{\rho_{14}{\left(t \right)} \left|{b_{0}}\right|^{2}}{2} - \frac{\rho_{14}{\left(t \right)} \left|{b_{1}}\right|^{2}}{2}\right) + \gamma_{--} \left(- \frac{\rho_{14}{\left(t \right)} \left|{b_{0}}\right|^{2}}{2} - \frac{\rho_{14}{\left(t \right)} \left|{b_{1}}\right|^{2}}{2}\right) + 8 i a \rho_{14}{\left(t \right)} + \left(- b_{0} \rho_{5}{\left(t \right)} \overline{b_{1}} - \rho_{12}{\left(t \right)} \overline{b_{0}} \overline{b_{1}} + \rho_{8}{\left(t \right)} \overline{b_{1}}^{2} + \rho_{9}{\left(t

In [33]:
for i in eqs:
    display(i)

Eq(Derivative(rho_1(t), t), \gamma_{++}*(b0*rho_7(t)*conjugate(b1) + b1*rho_10(t)*conjugate(b0) + rho_11(t)*Abs(b1)**2 + rho_6(t)*Abs(b0)**2) + \gamma_{--}*(-rho_1(t)*Abs(b0)**2 - rho_1(t)*Abs(b1)**2))

Eq(Derivative(rho_2(t), t), \gamma_{++}*(-rho_2(t)*Abs(b0)**2/2 - rho_2(t)*Abs(b1)**2/2) + \gamma_{--}*(-rho_2(t)*Abs(b0)**2/2 - rho_2(t)*Abs(b1)**2/2) + 8*I*a*rho_2(t) + (b0**2*rho_5(t) + b0*b1*rho_9(t) - b0*rho_8(t)*conjugate(b1) - rho_12(t)*Abs(b1)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_3(t), t), \gamma_{++}*(-rho_3(t)*Abs(b0)**2/2 - rho_3(t)*Abs(b1)**2/2) + \gamma_{--}*(-rho_3(t)*Abs(b0)**2/2 - rho_3(t)*Abs(b1)**2/2) + 8*I*a*rho_3(t) + (b0*b1*rho_5(t) + b1**2*rho_9(t) + b1*rho_12(t)*conjugate(b0) + rho_8(t)*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_4(t), t), \gamma_{++}*(b0**2*rho_7(t) + b0*b1*rho_11(t) - b0*b1*rho_6(t) - b1**2*rho_10(t)) + \gamma_{--}*(-rho_4(t)*Abs(b0)**2 - rho_4(t)*Abs(b1)**2))

Eq(Derivative(rho_5(t), t), \gamma_{++}*(-rho_5(t)*Abs(b0)**2/2 - rho_5(t)*Abs(b1)**2/2) + \gamma_{+-}*(-b1*rho_14(t)*conjugate(b0) - rho_15(t)*Abs(b1)**2 + rho_2(t)*conjugate(b0)**2 + rho_3(t)*conjugate(b0)*conjugate(b1)) + \gamma_{--}*(-rho_5(t)*Abs(b0)**2/2 - rho_5(t)*Abs(b1)**2/2) - 8*I*a*rho_5(t))

...

Eq(Derivative(rho_12(t), t), \gamma_{++}*(-rho_12(t)*Abs(b0)**2/2 - rho_12(t)*Abs(b1)**2/2) + \gamma_{+-}*(b0**2*rho_15(t) - b0*b1*rho_14(t) + b0*rho_3(t)*conjugate(b1) - rho_2(t)*Abs(b1)**2) + \gamma_{--}*(-rho_12(t)*Abs(b0)**2/2 - rho_12(t)*Abs(b1)**2/2) - 8*I*a*rho_12(t))

Eq(Derivative(rho_13(t), t), \gamma_{++}*(rho_10(t)*conjugate(b0)**2 + rho_11(t)*conjugate(b0)*conjugate(b1) - rho_6(t)*conjugate(b0)*conjugate(b1) - rho_7(t)*conjugate(b1)**2) + \gamma_{--}*(-rho_13(t)*Abs(b0)**2 - rho_13(t)*Abs(b1)**2))

Eq(Derivative(rho_14(t), t), \gamma_{++}*(-rho_14(t)*Abs(b0)**2/2 - rho_14(t)*Abs(b1)**2/2) + \gamma_{--}*(-rho_14(t)*Abs(b0)**2/2 - rho_14(t)*Abs(b1)**2/2) + 8*I*a*rho_14(t) + (-b0*rho_5(t)*conjugate(b1) - rho_12(t)*conjugate(b0)*conjugate(b1) + rho_8(t)*conjugate(b1)**2 + rho_9(t)*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_15(t), t), \gamma_{++}*(-rho_15(t)*Abs(b0)**2/2 - rho_15(t)*Abs(b1)**2/2) + \gamma_{--}*(-rho_15(t)*Abs(b0)**2/2 - rho_15(t)*Abs(b1)**2/2) + 8*I*a*rho_15(t) + (b1*rho_9(t)*conjugate(b0) + rho_12(t)*conjugate(b0)**2 - rho_5(t)*Abs(b1)**2 - rho_8(t)*conjugate(b0)*conjugate(b1))*conjugate(\gamma_{+-}))

Eq(Derivative(rho_16(t), t), \gamma_{++}*(-b0*rho_7(t)*conjugate(b1) - b1*rho_10(t)*conjugate(b0) + rho_11(t)*Abs(b0)**2 + rho_6(t)*Abs(b1)**2) + \gamma_{--}*(-rho_16(t)*Abs(b0)**2 - rho_16(t)*Abs(b1)**2))

Then We make $\gamma_{--}=0$ as we consider zero temperature $T=0$

In [34]:
(eqs[0].rhs + eqs[5].rhs + eqs[10].rhs + eqs[15].rhs ).subs(gmm,0).expand()

0

In [35]:
for i in eqs:
    display(i.subs(gmm,0))

Eq(Derivative(rho_1(t), t), \gamma_{++}*(b0*rho_7(t)*conjugate(b1) + b1*rho_10(t)*conjugate(b0) + rho_11(t)*Abs(b1)**2 + rho_6(t)*Abs(b0)**2))

Eq(Derivative(rho_2(t), t), \gamma_{++}*(-rho_2(t)*Abs(b0)**2/2 - rho_2(t)*Abs(b1)**2/2) + 8*I*a*rho_2(t) + (b0**2*rho_5(t) + b0*b1*rho_9(t) - b0*rho_8(t)*conjugate(b1) - rho_12(t)*Abs(b1)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_3(t), t), \gamma_{++}*(-rho_3(t)*Abs(b0)**2/2 - rho_3(t)*Abs(b1)**2/2) + 8*I*a*rho_3(t) + (b0*b1*rho_5(t) + b1**2*rho_9(t) + b1*rho_12(t)*conjugate(b0) + rho_8(t)*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_4(t), t), \gamma_{++}*(b0**2*rho_7(t) + b0*b1*rho_11(t) - b0*b1*rho_6(t) - b1**2*rho_10(t)))

Eq(Derivative(rho_5(t), t), \gamma_{++}*(-rho_5(t)*Abs(b0)**2/2 - rho_5(t)*Abs(b1)**2/2) + \gamma_{+-}*(-b1*rho_14(t)*conjugate(b0) - rho_15(t)*Abs(b1)**2 + rho_2(t)*conjugate(b0)**2 + rho_3(t)*conjugate(b0)*conjugate(b1)) - 8*I*a*rho_5(t))

...

Eq(Derivative(rho_12(t), t), \gamma_{++}*(-rho_12(t)*Abs(b0)**2/2 - rho_12(t)*Abs(b1)**2/2) + \gamma_{+-}*(b0**2*rho_15(t) - b0*b1*rho_14(t) + b0*rho_3(t)*conjugate(b1) - rho_2(t)*Abs(b1)**2) - 8*I*a*rho_12(t))

Eq(Derivative(rho_13(t), t), \gamma_{++}*(rho_10(t)*conjugate(b0)**2 + rho_11(t)*conjugate(b0)*conjugate(b1) - rho_6(t)*conjugate(b0)*conjugate(b1) - rho_7(t)*conjugate(b1)**2))

Eq(Derivative(rho_14(t), t), \gamma_{++}*(-rho_14(t)*Abs(b0)**2/2 - rho_14(t)*Abs(b1)**2/2) + 8*I*a*rho_14(t) + (-b0*rho_5(t)*conjugate(b1) - rho_12(t)*conjugate(b0)*conjugate(b1) + rho_8(t)*conjugate(b1)**2 + rho_9(t)*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_15(t), t), \gamma_{++}*(-rho_15(t)*Abs(b0)**2/2 - rho_15(t)*Abs(b1)**2/2) + 8*I*a*rho_15(t) + (b1*rho_9(t)*conjugate(b0) + rho_12(t)*conjugate(b0)**2 - rho_5(t)*Abs(b1)**2 - rho_8(t)*conjugate(b0)*conjugate(b1))*conjugate(\gamma_{+-}))

Eq(Derivative(rho_16(t), t), \gamma_{++}*(-b0*rho_7(t)*conjugate(b1) - b1*rho_10(t)*conjugate(b0) + rho_11(t)*Abs(b0)**2 + rho_6(t)*Abs(b1)**2))

Then at $T=0$, and in the Schrodinger picture (So that there is no phase and $\gamma$s are real). We have $\gamma_{+-}=\gamma_{-+}=\frac{\gamma_{++}}{2}$

In [36]:
for i in eqs:
    display(i.subs(gmm,0).subs(gpm,gpp/2).subs(conjugate(gpm),gpp/2))

Eq(Derivative(rho_1(t), t), \gamma_{++}*(b0*rho_7(t)*conjugate(b1) + b1*rho_10(t)*conjugate(b0) + rho_11(t)*Abs(b1)**2 + rho_6(t)*Abs(b0)**2))

Eq(Derivative(rho_2(t), t), \gamma_{++}*(-rho_2(t)*Abs(b0)**2/2 - rho_2(t)*Abs(b1)**2/2) + \gamma_{++}*(b0**2*rho_5(t) + b0*b1*rho_9(t) - b0*rho_8(t)*conjugate(b1) - rho_12(t)*Abs(b1)**2)/2 + 8*I*a*rho_2(t))

Eq(Derivative(rho_3(t), t), \gamma_{++}*(-rho_3(t)*Abs(b0)**2/2 - rho_3(t)*Abs(b1)**2/2) + \gamma_{++}*(b0*b1*rho_5(t) + b1**2*rho_9(t) + b1*rho_12(t)*conjugate(b0) + rho_8(t)*Abs(b0)**2)/2 + 8*I*a*rho_3(t))

Eq(Derivative(rho_4(t), t), \gamma_{++}*(b0**2*rho_7(t) + b0*b1*rho_11(t) - b0*b1*rho_6(t) - b1**2*rho_10(t)))

Eq(Derivative(rho_5(t), t), \gamma_{++}*(-rho_5(t)*Abs(b0)**2/2 - rho_5(t)*Abs(b1)**2/2) + \gamma_{++}*(-b1*rho_14(t)*conjugate(b0) - rho_15(t)*Abs(b1)**2 + rho_2(t)*conjugate(b0)**2 + rho_3(t)*conjugate(b0)*conjugate(b1))/2 - 8*I*a*rho_5(t))

...

Eq(Derivative(rho_12(t), t), \gamma_{++}*(-rho_12(t)*Abs(b0)**2/2 - rho_12(t)*Abs(b1)**2/2) + \gamma_{++}*(b0**2*rho_15(t) - b0*b1*rho_14(t) + b0*rho_3(t)*conjugate(b1) - rho_2(t)*Abs(b1)**2)/2 - 8*I*a*rho_12(t))

Eq(Derivative(rho_13(t), t), \gamma_{++}*(rho_10(t)*conjugate(b0)**2 + rho_11(t)*conjugate(b0)*conjugate(b1) - rho_6(t)*conjugate(b0)*conjugate(b1) - rho_7(t)*conjugate(b1)**2))

Eq(Derivative(rho_14(t), t), \gamma_{++}*(-rho_14(t)*Abs(b0)**2/2 - rho_14(t)*Abs(b1)**2/2) + \gamma_{++}*(-b0*rho_5(t)*conjugate(b1) - rho_12(t)*conjugate(b0)*conjugate(b1) + rho_8(t)*conjugate(b1)**2 + rho_9(t)*Abs(b0)**2)/2 + 8*I*a*rho_14(t))

Eq(Derivative(rho_15(t), t), \gamma_{++}*(-rho_15(t)*Abs(b0)**2/2 - rho_15(t)*Abs(b1)**2/2) + \gamma_{++}*(b1*rho_9(t)*conjugate(b0) + rho_12(t)*conjugate(b0)**2 - rho_5(t)*Abs(b1)**2 - rho_8(t)*conjugate(b0)*conjugate(b1))/2 + 8*I*a*rho_15(t))

Eq(Derivative(rho_16(t), t), \gamma_{++}*(-b0*rho_7(t)*conjugate(b1) - b1*rho_10(t)*conjugate(b0) + rho_11(t)*Abs(b0)**2 + rho_6(t)*Abs(b1)**2))

In [38]:
eqs2=[i.subs(gmm,0) for i in eqs]

At this point, we are still preserving the trace

In [39]:
Eq(eqs2[0].lhs + eqs2[5].lhs + eqs2[10].lhs + eqs2[15].lhs,(eqs2[0].rhs + eqs2[5].rhs + eqs2[10].rhs + eqs2[15].rhs ).expand())

Eq(Derivative(rho_1(t), t) + Derivative(rho_11(t), t) + Derivative(rho_16(t), t) + Derivative(rho_6(t), t), 0)

I first solve the really easy ones: ($\rho_{10},\rho_{11},\rho_{7},\rho_{6}$)

In [40]:
sols_easy=dsolve([eqs2[i] for i in (5,6,9,10)])
sols_subs={i.lhs:i.rhs for i in sols_easy}

In [42]:
for i in sols_easy:
    print(latex(i))

\rho_{6}{\left(t \right)} = C_{1} e^{- t \left(\gamma_{++} \left|{b_{0}}\right|^{2} + \gamma_{++} \left|{b_{1}}\right|^{2}\right)}
\rho_{7}{\left(t \right)} = C_{2} e^{- t \left(\gamma_{++} \left|{b_{0}}\right|^{2} + \gamma_{++} \left|{b_{1}}\right|^{2}\right)}
\rho_{10}{\left(t \right)} = C_{3} e^{- t \left(\gamma_{++} \left|{b_{0}}\right|^{2} + \gamma_{++} \left|{b_{1}}\right|^{2}\right)}
\rho_{11}{\left(t \right)} = C_{4} e^{- t \left(\gamma_{++} \left|{b_{0}}\right|^{2} + \gamma_{++} \left|{b_{1}}\right|^{2}\right)}


In [46]:
eqs3_full=[i.subs(sols_subs) for i in eqs2]
eqs3=[i.simplify() for i in eqs3_full if i.simplify()!=True ]
for i in eqs3:
    print(latex(i.subs(abs(b[0])**2 + abs(b[1])**2,c[1] )))

\frac{d}{d t} \rho_{1}{\left(t \right)} = \gamma_{++} \left(C_{1} \left|{b_{0}}\right|^{2} + C_{2} b_{0} \overline{b_{1}} + C_{3} b_{1} \overline{b_{0}} + C_{4} \left|{b_{1}}\right|^{2}\right) e^{- \gamma_{++} c_{1} t}
\frac{d}{d t} \rho_{2}{\left(t \right)} = - \frac{\gamma_{++} c_{1} \rho_{2}{\left(t \right)}}{2} + 8 i a \rho_{2}{\left(t \right)} + \left(b_{0}^{2} \rho_{5}{\left(t \right)} + b_{0} b_{1} \rho_{9}{\left(t \right)} - b_{0} \rho_{8}{\left(t \right)} \overline{b_{1}} - \rho_{12}{\left(t \right)} \left|{b_{1}}\right|^{2}\right) \overline{\gamma_{+-}}
\frac{d}{d t} \rho_{3}{\left(t \right)} = - \frac{\gamma_{++} c_{1} \rho_{3}{\left(t \right)}}{2} + 8 i a \rho_{3}{\left(t \right)} + \left(b_{0} b_{1} \rho_{5}{\left(t \right)} + b_{1}^{2} \rho_{9}{\left(t \right)} + b_{1} \rho_{12}{\left(t \right)} \overline{b_{0}} + \rho_{8}{\left(t \right)} \left|{b_{0}}\right|^{2}\right) \overline{\gamma_{+-}}
\frac{d}{d t} \rho_{4}{\left(t \right)} = - \gamma_{++} \left(C_{1} b_{0} b_{1}

The trace is still preserved as expected

In [47]:
Eq(eqs2[0].lhs + eqs2[5].lhs + eqs2[10].lhs + eqs2[15].lhs,(eqs3_full[0].rhs + eqs3_full[5].rhs + eqs3_full[10].rhs + eqs3_full[15].rhs ).expand())

Eq(Derivative(rho_1(t), t) + Derivative(rho_11(t), t) + Derivative(rho_16(t), t) + Derivative(rho_6(t), t), 0)

In [114]:
sols_easy2=dsolve([eqs3[i] for i in (0,3,8,11)])
sols_subs2={i.lhs:i.rhs for i in sols_easy2}

In [115]:
for i in sols_easy2:
    print(latex(i.simplify().subs(abs(b[0])**2 + abs(b[1])**2,c[1] ).simplify()))

\rho_{1}{\left(t \right)} = \frac{\left(- C_{1} \left|{b_{0}}\right|^{2} - C_{2} b_{0} \overline{b_{1}} - C_{3} b_{1} \overline{b_{0}} - C_{4} \left|{b_{1}}\right|^{2} + C_{5} c_{1} e^{\gamma_{++} c_{1} t}\right) e^{- \gamma_{++} c_{1} t}}{c_{1}}
\rho_{4}{\left(t \right)} = \frac{\left(C_{1} b_{0} b_{1} - C_{2} b_{0}^{2} + C_{3} b_{1}^{2} - C_{4} b_{0} b_{1} + C_{6} c_{1} e^{\gamma_{++} c_{1} t}\right) e^{- \gamma_{++} c_{1} t}}{c_{1}}
\rho_{13}{\left(t \right)} = \frac{\left(C_{1} \overline{b_{0}} \overline{b_{1}} + C_{2} \overline{b_{1}}^{2} - C_{3} \overline{b_{0}}^{2} - C_{4} \overline{b_{0}} \overline{b_{1}} + C_{7} c_{1} e^{\gamma_{++} c_{1} t}\right) e^{- \gamma_{++} c_{1} t}}{c_{1}}
\rho_{16}{\left(t \right)} = \frac{\left(- C_{1} \left|{b_{1}}\right|^{2} + C_{2} b_{0} \overline{b_{1}} + C_{3} b_{1} \overline{b_{0}} - C_{4} \left|{b_{0}}\right|^{2} + C_{8} c_{1} e^{\gamma_{++} c_{1} t}\right) e^{- \gamma_{++} c_{1} t}}{c_{1}}


In [117]:
eqs4_full=[i.subs(sols_subs).subs(sols_subs2) for i in eqs2]
eqs4=[Eq(i.lhs,i.simplify().subs(abs(b[0])**2 + abs(b[1])**2,c[1] ).rhs.collect(rhof[1](t)).collect(rhof[2](t)).collect(rhof[4](t)).collect(rhof[7](t)).collect(rhof[8](t)).collect(rhof[11](t)).collect(rhof[13](t)).collect(rhof[14](t))) for i in eqs4_full if i.simplify()!=True ]
eqs4=[i.subs(-gpp*c[1]/2 + 8*I*a,c[2]).subs(-gpp*c[1]/2 - 8*I*a,conjugate(c[2])) for i in eqs4]
for i in eqs4:
    #print(latex(i)+"\\\\")
    display(i)

Eq(Derivative(rho_2(t), t), c2*rho_2(t) + (b0**2*rho_5(t) + b0*b1*rho_9(t) - b0*rho_8(t)*conjugate(b1) - rho_12(t)*Abs(b1)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_3(t), t), c2*rho_3(t) + (b0*b1*rho_5(t) + b1**2*rho_9(t) + b1*rho_12(t)*conjugate(b0) + rho_8(t)*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_5(t), t), -\gamma_{+-}*(b1*rho_14(t)*conjugate(b0) + rho_15(t)*Abs(b1)**2 - rho_2(t)*conjugate(b0)**2 - rho_3(t)*conjugate(b0)*conjugate(b1)) + rho_5(t)*conjugate(c2))

Eq(Derivative(rho_8(t), t), -\gamma_{+-}*(b0*b1*rho_15(t) - b1**2*rho_14(t) + b1*rho_2(t)*conjugate(b0) - rho_3(t)*Abs(b0)**2) + rho_8(t)*conjugate(c2))

Eq(Derivative(rho_9(t), t), \gamma_{+-}*(b0*rho_15(t)*conjugate(b1) + rho_14(t)*Abs(b0)**2 + rho_2(t)*conjugate(b0)*conjugate(b1) + rho_3(t)*conjugate(b1)**2) + rho_9(t)*conjugate(c2))

Eq(Derivative(rho_12(t), t), \gamma_{+-}*(b0**2*rho_15(t) - b0*b1*rho_14(t) + b0*rho_3(t)*conjugate(b1) - rho_2(t)*Abs(b1)**2) + rho_12(t)*conjugate(c2))

Eq(Derivative(rho_14(t), t), c2*rho_14(t) - (b0*rho_5(t)*conjugate(b1) + rho_12(t)*conjugate(b0)*conjugate(b1) - rho_8(t)*conjugate(b1)**2 - rho_9(t)*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_15(t), t), c2*rho_15(t) + (b1*rho_9(t)*conjugate(b0) + rho_12(t)*conjugate(b0)**2 - rho_5(t)*Abs(b1)**2 - rho_8(t)*conjugate(b0)*conjugate(b1))*conjugate(\gamma_{+-}))

In [205]:
eqs4=[Eq(0,i.simplify().subs(abs(b[0])**2 + abs(b[1])**2,c[1] ).rhs.collect(rhof[1](t)).collect(rhof[2](t)).collect(rhof[4](t)).collect(rhof[7](t)).collect(rhof[8](t)).collect(rhof[11](t)).collect(rhof[13](t)).collect(rhof[14](t))) for i in eqs4_full if i.simplify()!=True ]


In [ ]:
A.nullspace()

In [157]:
sys=[i.rhs for i in eqs4]

In [161]:
remaining=[mrhof[i] for i in (1,2,4,7,8,11,13,14)]
remaining

[rho_2(t),
 rho_3(t),
 rho_5(t),
 rho_8(t),
 rho_9(t),
 rho_12(t),
 rho_14(t),
 rho_15(t)]

In [192]:
A,V=linear_eq_to_matrix(sys, remaining)


In [203]:
A.subs(b[0],b[1]).subs(b[1]*conjugate(b[1]),abs(b[1])**2)

Matrix([
[                          c2,                            0,       b1**2*conjugate(\gamma_{+-}),       -conjugate(\gamma_{+-})*Abs(b1)**2,      b1**2*conjugate(\gamma_{+-}),       -conjugate(\gamma_{+-})*Abs(b1)**2,                       0,                       0],
[                           0,                           c2,       b1**2*conjugate(\gamma_{+-}),        conjugate(\gamma_{+-})*Abs(b1)**2,      b1**2*conjugate(\gamma_{+-}),        conjugate(\gamma_{+-})*Abs(b1)**2,                       0,                       0],
[\gamma_{+-}*conjugate(b1)**2, \gamma_{+-}*conjugate(b1)**2,                      conjugate(c2),                                        0,                                 0,                                        0, -\gamma_{+-}*Abs(b1)**2, -\gamma_{+-}*Abs(b1)**2],
[     -\gamma_{+-}*Abs(b1)**2,       \gamma_{+-}*Abs(b1)**2,                                  0,                            conjugate(c2),                                 0,                 

In [204]:
P, D = A.subs(b[0],b[1]).subs(b[1]*conjugate(b[1]),abs(b[1])**2).diagonalize()

KeyboardInterrupt: 

In [ ]:
P

In [167]:
V

Matrix([
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0]])

In [125]:
rhof[0](t)

rho_1(t)

In [140]:
rhof[1](t)

rho_2(t)

In [154]:
A = Matrix([[eq.rhs.coeff(var) for var in mrhof] for eq in eqs4])
b = Matrix([-eq.rhs + sum(eq.lhs.coeff(var) * var for var in mrhof) for eq in eqs4])

In [155]:
A

Matrix([
[0, c2,  0, 0,             0, 0, 0,             0,             0, 0, 0,             0, 0,  0,  0, 0],
[0,  0, c2, 0,             0, 0, 0,             0,             0, 0, 0,             0, 0,  0,  0, 0],
[0,  0,  0, 0, conjugate(c2), 0, 0,             0,             0, 0, 0,             0, 0,  0,  0, 0],
[0,  0,  0, 0,             0, 0, 0, conjugate(c2),             0, 0, 0,             0, 0,  0,  0, 0],
[0,  0,  0, 0,             0, 0, 0,             0, conjugate(c2), 0, 0,             0, 0,  0,  0, 0],
[0,  0,  0, 0,             0, 0, 0,             0,             0, 0, 0, conjugate(c2), 0,  0,  0, 0],
[0,  0,  0, 0,             0, 0, 0,             0,             0, 0, 0,             0, 0, c2,  0, 0],
[0,  0,  0, 0,             0, 0, 0,             0,             0, 0, 0,             0, 0,  0, c2, 0]])

In [156]:
b

Matrix([
[                                  -c2*rho_2(t) - (b0**2*rho_5(t) + b0*b1*rho_9(t) - b0*rho_8(t)*conjugate(b1) - rho_12(t)*Abs(b1)**2)*conjugate(\gamma_{+-})],
[                                  -c2*rho_3(t) - (b0*b1*rho_5(t) + b1**2*rho_9(t) + b1*rho_12(t)*conjugate(b0) + rho_8(t)*Abs(b0)**2)*conjugate(\gamma_{+-})],
[ \gamma_{+-}*(b1*rho_14(t)*conjugate(b0) + rho_15(t)*Abs(b1)**2 - rho_2(t)*conjugate(b0)**2 - rho_3(t)*conjugate(b0)*conjugate(b1)) - rho_5(t)*conjugate(c2)],
[                                  \gamma_{+-}*(b0*b1*rho_15(t) - b1**2*rho_14(t) + b1*rho_2(t)*conjugate(b0) - rho_3(t)*Abs(b0)**2) - rho_8(t)*conjugate(c2)],
[-\gamma_{+-}*(b0*rho_15(t)*conjugate(b1) + rho_14(t)*Abs(b0)**2 + rho_2(t)*conjugate(b0)*conjugate(b1) + rho_3(t)*conjugate(b1)**2) - rho_9(t)*conjugate(c2)],
[                                -\gamma_{+-}*(b0**2*rho_15(t) - b0*b1*rho_14(t) + b0*rho_3(t)*conjugate(b1) - rho_2(t)*Abs(b1)**2) - rho_12(t)*conjugate(c2)],
[-c2*rho_14(t) + (b0*rho_5(t)*c

In [101]:
for i in eqs4:
    display(i)

Eq(Derivative(rho_2(t), t), c2*rho_2(t) + (b0**2*conjugate(rho_2(t)) + b0*b1*conjugate(rho_3(t)) - b0*rho_8(t)*conjugate(b1) - rho_12(t)*Abs(b1)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_3(t), t), c2*rho_3(t) + (b0*b1*conjugate(rho_2(t)) + b1**2*conjugate(rho_3(t)) + b1*rho_12(t)*conjugate(b0) + rho_8(t)*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(conjugate(rho_2(t)), t), -\gamma_{+-}*(b1*conjugate(b0)*conjugate(rho_8(t)) - rho_2(t)*conjugate(b0)**2 - rho_3(t)*conjugate(b0)*conjugate(b1) + conjugate(rho_12(t))*Abs(b1)**2) + conjugate(c2)*conjugate(rho_2(t)))

Eq(Derivative(rho_8(t), t), -\gamma_{+-}*(b0*b1*conjugate(rho_12(t)) - b1**2*conjugate(rho_8(t)) + b1*rho_2(t)*conjugate(b0) - rho_3(t)*Abs(b0)**2) + rho_8(t)*conjugate(c2))

Eq(Derivative(conjugate(rho_3(t)), t), \gamma_{+-}*(b0*conjugate(b1)*conjugate(rho_12(t)) + rho_2(t)*conjugate(b0)*conjugate(b1) + rho_3(t)*conjugate(b1)**2 + conjugate(rho_8(t))*Abs(b0)**2) + conjugate(c2)*conjugate(rho_3(t)))

Eq(Derivative(rho_12(t), t), \gamma_{+-}*(b0**2*conjugate(rho_12(t)) - b0*b1*conjugate(rho_8(t)) + b0*rho_3(t)*conjugate(b1) - rho_2(t)*Abs(b1)**2) + rho_12(t)*conjugate(c2))

Eq(Derivative(conjugate(rho_8(t)), t), c2*conjugate(rho_8(t)) - (b0*conjugate(b1)*conjugate(rho_2(t)) + rho_12(t)*conjugate(b0)*conjugate(b1) - rho_8(t)*conjugate(b1)**2 - conjugate(rho_3(t))*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(conjugate(rho_12(t)), t), c2*conjugate(rho_12(t)) + (b1*conjugate(b0)*conjugate(rho_3(t)) + rho_12(t)*conjugate(b0)**2 - rho_8(t)*conjugate(b0)*conjugate(b1) - conjugate(rho_2(t))*Abs(b1)**2)*conjugate(\gamma_{+-}))

In [98]:
(eqs4[0].rhs + eqs4[2].rhs).expand(xomplex=True).simplify()

-\gamma_{+-}*b1*conjugate(b0)*conjugate(rho_8(t)) + \gamma_{+-}*rho_2(t)*conjugate(b0)**2 + \gamma_{+-}*rho_3(t)*conjugate(b0)*conjugate(b1) - \gamma_{+-}*conjugate(rho_12(t))*Abs(b1)**2 + b0**2*conjugate(\gamma_{+-})*conjugate(rho_2(t)) + b0*b1*conjugate(\gamma_{+-})*conjugate(rho_3(t)) - b0*rho_8(t)*conjugate(\gamma_{+-})*conjugate(b1) + c2*rho_2(t) - rho_12(t)*conjugate(\gamma_{+-})*Abs(b1)**2 + conjugate(c2)*conjugate(rho_2(t))

In [111]:
sample=[eqs4[i] for i in (0,1,3,5)]
for i in eqs4:
    display(i)

Eq(Derivative(rho_2(t), t), c2*rho_2(t) + (b0**2*conjugate(rho_2(t)) + b0*b1*conjugate(rho_3(t)) - b0*rho_8(t)*conjugate(b1) - rho_12(t)*Abs(b1)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(rho_3(t), t), c2*rho_3(t) + (b0*b1*conjugate(rho_2(t)) + b1**2*conjugate(rho_3(t)) + b1*rho_12(t)*conjugate(b0) + rho_8(t)*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(conjugate(rho_2(t)), t), -\gamma_{+-}*(b1*conjugate(b0)*conjugate(rho_8(t)) - rho_2(t)*conjugate(b0)**2 - rho_3(t)*conjugate(b0)*conjugate(b1) + conjugate(rho_12(t))*Abs(b1)**2) + conjugate(c2)*conjugate(rho_2(t)))

Eq(Derivative(rho_8(t), t), -\gamma_{+-}*(b0*b1*conjugate(rho_12(t)) - b1**2*conjugate(rho_8(t)) + b1*rho_2(t)*conjugate(b0) - rho_3(t)*Abs(b0)**2) + rho_8(t)*conjugate(c2))

Eq(Derivative(conjugate(rho_3(t)), t), \gamma_{+-}*(b0*conjugate(b1)*conjugate(rho_12(t)) + rho_2(t)*conjugate(b0)*conjugate(b1) + rho_3(t)*conjugate(b1)**2 + conjugate(rho_8(t))*Abs(b0)**2) + conjugate(c2)*conjugate(rho_3(t)))

Eq(Derivative(rho_12(t), t), \gamma_{+-}*(b0**2*conjugate(rho_12(t)) - b0*b1*conjugate(rho_8(t)) + b0*rho_3(t)*conjugate(b1) - rho_2(t)*Abs(b1)**2) + rho_12(t)*conjugate(c2))

Eq(Derivative(conjugate(rho_8(t)), t), c2*conjugate(rho_8(t)) - (b0*conjugate(b1)*conjugate(rho_2(t)) + rho_12(t)*conjugate(b0)*conjugate(b1) - rho_8(t)*conjugate(b1)**2 - conjugate(rho_3(t))*Abs(b0)**2)*conjugate(\gamma_{+-}))

Eq(Derivative(conjugate(rho_12(t)), t), c2*conjugate(rho_12(t)) + (b1*conjugate(b0)*conjugate(rho_3(t)) + rho_12(t)*conjugate(b0)**2 - rho_8(t)*conjugate(b0)*conjugate(b1) - conjugate(rho_2(t))*Abs(b1)**2)*conjugate(\gamma_{+-}))

In [110]:
(eqs4[0].rhs  + eqs4[2].rhs).expand(complex=True).simplify()/2

re(\gamma_{+-})*re(b0)**2*re(rho_2(t)) + re(\gamma_{+-})*re(b0)*re(b1)*re(rho_3(t)) - re(\gamma_{+-})*re(b0)*re(b1)*re(rho_8(t)) + 2*re(\gamma_{+-})*re(b0)*im(b0)*im(rho_2(t)) + re(\gamma_{+-})*re(b0)*im(b1)*im(rho_3(t)) - re(\gamma_{+-})*re(b0)*im(b1)*im(rho_8(t)) - re(\gamma_{+-})*re(b1)**2*re(rho_12(t)) + re(\gamma_{+-})*re(b1)*im(b0)*im(rho_3(t)) + re(\gamma_{+-})*re(b1)*im(b0)*im(rho_8(t)) - re(\gamma_{+-})*re(rho_12(t))*im(b1)**2 - re(\gamma_{+-})*re(rho_2(t))*im(b0)**2 - re(\gamma_{+-})*re(rho_3(t))*im(b0)*im(b1) - re(\gamma_{+-})*re(rho_8(t))*im(b0)*im(b1) - re(b0)**2*im(\gamma_{+-})*im(rho_2(t)) - re(b0)*re(b1)*im(\gamma_{+-})*im(rho_3(t)) - re(b0)*re(b1)*im(\gamma_{+-})*im(rho_8(t)) + 2*re(b0)*re(rho_2(t))*im(\gamma_{+-})*im(b0) + re(b0)*re(rho_3(t))*im(\gamma_{+-})*im(b1) + re(b0)*re(rho_8(t))*im(\gamma_{+-})*im(b1) - re(b1)**2*im(\gamma_{+-})*im(rho_12(t)) + re(b1)*re(rho_3(t))*im(\gamma_{+-})*im(b0) - re(b1)*re(rho_8(t))*im(\gamma_{+-})*im(b0) + re(c2)*re(rho_2(t)) + im(\g

In [90]:
sols_easy2=dsolve([eqs4[i] for i in (0,1,3,5)])


NotImplementedError: 

The set of equation does preserve the trace, we made a mistake in person

In [62]:
Eq(eqs2[0].lhs + eqs2[5].lhs + eqs2[10].lhs + eqs2[15].lhs,(eqs4_full[0].rhs + eqs4_full[5].rhs + eqs4_full[10].rhs + eqs4_full[15].rhs ).expand())

Eq(Derivative(rho_1(t), t) + Derivative(rho_11(t), t) + Derivative(rho_16(t), t) + Derivative(rho_6(t), t), 0)

I cannot solve analytically for the general case due to the degree of the polynomial, I can then use  specific values of $b_{0}$ and $b_{1}$ so that I can express it as radicals ( I did not try mathematica though but even If I do it would be a root object). But I can solve the diagonal (But that does not help because the diagonal works)


In [63]:
for i in sols_easy:
    display(i)

Eq(rho_6(t), C1*exp(-t*(\gamma_{++}*Abs(b0)**2 + \gamma_{++}*Abs(b1)**2)))

Eq(rho_7(t), C2*exp(-t*(\gamma_{++}*Abs(b0)**2 + \gamma_{++}*Abs(b1)**2)))

Eq(rho_10(t), C3*exp(-t*(\gamma_{++}*Abs(b0)**2 + \gamma_{++}*Abs(b1)**2)))

Eq(rho_11(t), C4*exp(-t*(\gamma_{++}*Abs(b0)**2 + \gamma_{++}*Abs(b1)**2)))

Send references to Shishir Khandelwal on this!

In [64]:
for i in sols_easy2:
    display(i)

Eq(rho_1(t), C5 - (C1*Abs(b0)**2 + C2*b0*conjugate(b1) + C3*b1*conjugate(b0) + C4*Abs(b1)**2)*exp(-t*(\gamma_{++}*Abs(b0)**2 + \gamma_{++}*Abs(b1)**2))/(Abs(b0)**2 + Abs(b1)**2))

Eq(rho_4(t), C6 + (C1*b0*b1 - C2*b0**2 + C3*b1**2 - C4*b0*b1)*exp(-t*(\gamma_{++}*Abs(b0)**2 + \gamma_{++}*Abs(b1)**2))/(Abs(b0)**2 + Abs(b1)**2))

Eq(rho_13(t), C7 + (C1*conjugate(b0)*conjugate(b1) + C2*conjugate(b1)**2 - C3*conjugate(b0)**2 - C4*conjugate(b0)*conjugate(b1))*exp(-t*(\gamma_{++}*Abs(b0)**2 + \gamma_{++}*Abs(b1)**2))/(Abs(b0)**2 + Abs(b1)**2))

Eq(rho_16(t), C8 - (C1*Abs(b1)**2 - C2*b0*conjugate(b1) - C3*b1*conjugate(b0) + C4*Abs(b0)**2)*exp(-t*(\gamma_{++}*Abs(b0)**2 + \gamma_{++}*Abs(b1)**2))/(Abs(b0)**2 + Abs(b1)**2))

And No contradiction because the trace is preserved, but the time dependent trace does have an issue because

In [65]:
trace_pres=Eq(sols_easy2[0].lhs + sols_easy[0].lhs +sols_easy[-1].lhs+sols_easy2[-1].lhs ,sols_easy2[0].rhs + sols_easy[0].rhs+sols_easy[-1].rhs+sols_easy2[-1].rhs)

In [68]:
trace_pres.subs(t,0).simplify()

Eq(C5 + C8, rho_1(0) + rho_11(0) + rho_16(0) + rho_6(0))

Which simplifies to

In [69]:
Eq(trace_pres.lhs,trace_pres.rhs.simplify())

Eq(rho_1(t) + rho_11(t) + rho_16(t) + rho_6(t), C5 + C8)

Which is odd, and perhaps inconsistent

In [70]:
final=dict((v, k) for k, v in to_funcs.items())


So I then try for the steady state

In [71]:
ss=[Eq(i.rhs,0).subs(final) for i in eqs2]
for i in ss:
    display(i)

Eq(\gamma_{++}*(b0*rho_7*conjugate(b1) + b1*rho_10*conjugate(b0) + rho_11*Abs(b1)**2 + rho_6*Abs(b0)**2), 0)

Eq(\gamma_{++}*(-rho_2*Abs(b0)**2/2 - rho_2*Abs(b1)**2/2) + 8*I*a*rho_2 + (b0**2*rho_5 + b0*b1*rho_9 - b0*rho_8*conjugate(b1) - rho_12*Abs(b1)**2)*conjugate(\gamma_{+-}), 0)

Eq(\gamma_{++}*(-rho_3*Abs(b0)**2/2 - rho_3*Abs(b1)**2/2) + 8*I*a*rho_3 + (b0*b1*rho_5 + b1**2*rho_9 + b1*rho_12*conjugate(b0) + rho_8*Abs(b0)**2)*conjugate(\gamma_{+-}), 0)

Eq(\gamma_{++}*(b0**2*rho_7 + b0*b1*rho_11 - b0*b1*rho_6 - b1**2*rho_10), 0)

Eq(\gamma_{++}*(-rho_5*Abs(b0)**2/2 - rho_5*Abs(b1)**2/2) + \gamma_{+-}*(-b1*rho_14*conjugate(b0) - rho_15*Abs(b1)**2 + rho_2*conjugate(b0)**2 + rho_3*conjugate(b0)*conjugate(b1)) - 8*I*a*rho_5, 0)

...

Eq(\gamma_{++}*(-rho_12*Abs(b0)**2/2 - rho_12*Abs(b1)**2/2) + \gamma_{+-}*(b0**2*rho_15 - b0*b1*rho_14 + b0*rho_3*conjugate(b1) - rho_2*Abs(b1)**2) - 8*I*a*rho_12, 0)

Eq(\gamma_{++}*(rho_10*conjugate(b0)**2 + rho_11*conjugate(b0)*conjugate(b1) - rho_6*conjugate(b0)*conjugate(b1) - rho_7*conjugate(b1)**2), 0)

Eq(\gamma_{++}*(-rho_14*Abs(b0)**2/2 - rho_14*Abs(b1)**2/2) + 8*I*a*rho_14 + (-b0*rho_5*conjugate(b1) - rho_12*conjugate(b0)*conjugate(b1) + rho_8*conjugate(b1)**2 + rho_9*Abs(b0)**2)*conjugate(\gamma_{+-}), 0)

Eq(\gamma_{++}*(-rho_15*Abs(b0)**2/2 - rho_15*Abs(b1)**2/2) + 8*I*a*rho_15 + (b1*rho_9*conjugate(b0) + rho_12*conjugate(b0)**2 - rho_5*Abs(b1)**2 - rho_8*conjugate(b0)*conjugate(b1))*conjugate(\gamma_{+-}), 0)

Eq(\gamma_{++}*(-b0*rho_7*conjugate(b1) - b1*rho_10*conjugate(b0) + rho_11*Abs(b0)**2 + rho_6*Abs(b1)**2), 0)

From those equations obviously $\rho_{10},\rho_{6},\rho_{11} = 0$, Substitution then gives us no information on the diagonal, but no inconsistency as we puzzled about before

In [176]:
c= symbols('c0:8')

In [73]:
ss2=[Eq(i.rhs,0).subs(final).subs(rho[2,1],0).subs(rho[1,1],0).subs(rho[2,2],0)  for i  in eqs2]
ss2=[i.simplify() for i in ss2 if i.simplify()!=True ]
new_vars={gpp/2 * (abs(b[0])**2 + abs(b[1])**2 ):c[0]}
for i in ss2:
    display(i.subs(new_vars))

Eq(\gamma_{++}*b0*rho_7*conjugate(b1), 0)

Eq(8*I*a*rho_2 - c0*rho_2 + (b0**2*rho_5 + b0*b1*rho_9 - b0*rho_8*conjugate(b1) - rho_12*Abs(b1)**2)*conjugate(\gamma_{+-}), 0)

Eq(8*I*a*rho_3 - c0*rho_3 + (b0*b1*rho_5 + b1**2*rho_9 + b1*rho_12*conjugate(b0) + rho_8*Abs(b0)**2)*conjugate(\gamma_{+-}), 0)

Eq(\gamma_{++}*b0**2*rho_7, 0)

Eq(\gamma_{+-}*(b1*rho_14*conjugate(b0) + rho_15*Abs(b1)**2 - rho_2*conjugate(b0)**2 - rho_3*conjugate(b0)*conjugate(b1)) + 8*I*a*rho_5 + c0*rho_5, 0)

...

Eq(-\gamma_{+-}*(b0**2*rho_15 - b0*b1*rho_14 + b0*rho_3*conjugate(b1) - rho_2*Abs(b1)**2) + 8*I*a*rho_12 + c0*rho_12, 0)

Eq(\gamma_{++}*rho_7*conjugate(b1)**2, 0)

Eq(-8*I*a*rho_14 + c0*rho_14 + (b0*rho_5*conjugate(b1) + rho_12*conjugate(b0)*conjugate(b1) - rho_8*conjugate(b1)**2 - rho_9*Abs(b0)**2)*conjugate(\gamma_{+-}), 0)

Eq(8*I*a*rho_15 - c0*rho_15 + (b1*rho_9*conjugate(b0) + rho_12*conjugate(b0)**2 - rho_5*Abs(b1)**2 - rho_8*conjugate(b0)*conjugate(b1))*conjugate(\gamma_{+-}), 0)

Eq(\gamma_{++}*b0*rho_7*conjugate(b1), 0)

The problem here is that I have more unknowns that solutions, for exmaple equations $1$ and $3$ are just one equation,  same for each pair so I have 4 equations but I have 8 variables the real and the imaginary part of $\rho_{2},\rho_{3},\rho_{8},\rho_{12}$, the diagonal is also undetermined from this set of equations

In [1]:
from sympy.physics.quantum import Commutator
from sympy import *

In [72]:
rho = symbols('x_1:17')
#a=symbols('a')
rho = Matrix(
    [rho[0: 4],
     rho[4: 8],
     rho[8: 12],
     rho[12:]])
H = diag(-a,a,a,-a)#the times 20 is to rescale time

print(latex(rho))

\left[\begin{matrix}x_{1} & x_{2} & x_{3} & x_{4}\\x_{5} & x_{6} & x_{7} & x_{8}\\x_{9} & x_{10} & x_{11} & x_{12}\\x_{13} & x_{14} & x_{15} & x_{16}\end{matrix}\right]


In [68]:
rhoo,Hh=symbols("rho H",commutative=False)
rhoo

rho

In [69]:
print(latex(Eq(Commutator(Hh,rhoo),H*rho -rho*H,evaluate=False)))

\left[H,\rho\right] = \left[\begin{matrix}0 & - 2 a x_{2} & - 2 a x_{3} & 0\\2 a x_{5} & 0 & 0 & 2 a x_{8}\\2 a x_{9} & 0 & 0 & 2 a x_{12}\\0 & - 2 a x_{14} & - 2 a x_{15} & 0\end{matrix}\right]


For an operator to commute with $H$ $\rho_{2}= \rho_{3}=\rho_{8}=\rho_{12} = 0$

:::{tip}
Because the density matris is hermitian this means we also have $\rho_{5}=\rho_{9}=\rho_{15}=\rho_{14} =0$
:::

In [84]:
rho=rho.subs({rho[0,1]:0,rho[0,2]:0,rho[1,3]:0,rho[2,3]:0})
rho=rho.subs({rho[1,0]:0,rho[2,0]:0,rho[3,1]:0,rho[3,2]:0})
#rho=rho.subs({rho[12]:conjugate(rho[3]),rho[6]:conjugate(rho[9]),rho[15]:1-(rho[0]+rho[5]-rho[10])})
rho

Matrix([
[ x_1,    0,    0,  x_4],
[   0,  x_6,  x_7,    0],
[   0, x_10, x_11,    0],
[x_13,    0,    0, x_16]])

In [92]:
A=jumps[2*a]
A

Matrix([
[0,             b0,            b1, 0],
[0,              0,             0, 0],
[0,              0,             0, 0],
[0, -conjugate(b1), conjugate(b0), 0]])

In [93]:
A*rho-rho*A

Matrix([
[0,                         -b0*x_1 + b0*x_6 + b1*x_10 + x_4*conjugate(b1),                          b0*x_7 - b1*x_1 + b1*x_11 - x_4*conjugate(b0), 0],
[0,                                                                      0,                                                                      0, 0],
[0,                                                                      0,                                                                      0, 0],
[0, -b0*x_13 + x_10*conjugate(b0) + x_16*conjugate(b1) - x_6*conjugate(b1), -b1*x_13 + x_11*conjugate(b0) - x_16*conjugate(b0) - x_7*conjugate(b1), 0]])

In [94]:
print(latex(A*rho-rho*A))

\left[\begin{matrix}0 & - b_{0} x_{1} + b_{0} x_{6} + b_{1} x_{10} + x_{4} \overline{b_{1}} & b_{0} x_{7} - b_{1} x_{1} + b_{1} x_{11} - x_{4} \overline{b_{0}} & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & - b_{0} x_{13} + x_{10} \overline{b_{0}} + x_{16} \overline{b_{1}} - x_{6} \overline{b_{1}} & - b_{1} x_{13} + x_{11} \overline{b_{0}} - x_{16} \overline{b_{0}} - x_{7} \overline{b_{1}} & 0\end{matrix}\right]


So an operator that commutes with both needs to satisy

In [106]:
eqs_commutant=[Eq(i.simplify(),0) for i in flatten(A*rho-rho*A) if i!=0]
#eqs_commutant.extend([Eq(i.simplify(),0) for i in flatten(Dagger(A)*rho-rho*Dagger(A)) if i!=0])
eqs_commutant.extend([Eq(rho[0]+rho[5]+rho[10]+rho[15],1)])
for i in eqs_commutant:
    #print(latex(i))
    display(i)

Eq(-b0*x_1 + b0*x_6 + b1*x_10 + x_4*conjugate(b1), 0)

Eq(b0*x_7 - b1*x_1 + b1*x_11 - x_4*conjugate(b0), 0)

Eq(-b0*x_13 + x_10*conjugate(b0) + x_16*conjugate(b1) - x_6*conjugate(b1), 0)

Eq(-b1*x_13 + x_11*conjugate(b0) - x_16*conjugate(b0) - x_7*conjugate(b1), 0)

Eq(x_1 + x_11 + x_16 + x_6, 1)

I think this has infinitely many solutions as well!

In [107]:
rho[3]

x_4

In [108]:
solution = solve(eqs_commutant, (rho[0], rho[3], rho[5], rho[6], rho[9], rho[10], rho[12], rho[15]))


In [109]:
solution

{x_1: 1/2 - x_16,
 x_10: b0*x_13/conjugate(b0) - x_11*conjugate(b1)/conjugate(b0) - x_16*conjugate(b1)/conjugate(b0) + conjugate(b1)/(2*conjugate(b0)),
 x_4: -b0*b1*x_13/(conjugate(b0)*conjugate(b1)) - b1/(2*conjugate(b0)) + x_11*(b0/conjugate(b1) + b1/conjugate(b0)) + x_16*(-b0/conjugate(b1) + b1/conjugate(b0)),
 x_6: 1/2 - x_11,
 x_7: -b1*x_13/conjugate(b1) + x_11*conjugate(b0)/conjugate(b1) - x_16*conjugate(b0)/conjugate(b1)}